In [2]:
import time

# --- Boyer-Moore Bad Character heuristic only ---
def build_bad_char_table(pattern):
    bad_char = {}
    for i, ch in enumerate(pattern):
        bad_char[ch] = i
    return bad_char

def boyer_moore_bad_character(text, pattern):
    bad_char = build_bad_char_table(pattern)
    m = len(pattern)
    n = len(text)
    s = 0
    while s <= n - m:
        j = m - 1
        while j >= 0 and pattern[j] == text[s + j]:
            j -= 1
        if j < 0:
            return True
        else:
            shift = j - bad_char.get(text[s + j], -1)
            s += max(1, shift)
    return False

# --- Boyer-Moore Good Suffix heuristic only ---
def build_good_suffix_table(pattern):
    m = len(pattern)
    good_suffix = [0] * (m + 1)
    border_pos = [0] * (m + 1)

    i = m
    j = m + 1
    border_pos[i] = j

    while i > 0:
        while j <= m and (i == 0 or pattern[i - 1] != pattern[j - 1]):
            if good_suffix[j] == 0:
                good_suffix[j] = j - i
            j = border_pos[j]
        i -= 1
        j -= 1
        border_pos[i] = j

    j = border_pos[0]
    for i in range(m + 1):
        if good_suffix[i] == 0:
            good_suffix[i] = j
        if i == j:
            j = border_pos[j]

    return good_suffix

def boyer_moore_good_suffix(text, pattern):
    m = len(pattern)
    n = len(text)
    good_suffix = build_good_suffix_table(pattern)
    s = 0
    while s <= n - m:
        j = m - 1
        while j >= 0 and pattern[j] == text[s + j]:
            j -= 1
        if j < 0:
            return True
        else:
            s += good_suffix[j + 1]
    return False

# --- Boyer-Moore Combined heuristic ---
def boyer_moore_combined(text, pattern):
    bad_char = build_bad_char_table(pattern)
    good_suffix = build_good_suffix_table(pattern)
    m = len(pattern)
    n = len(text)
    s = 0
    while s <= n - m:
        j = m - 1
        while j >= 0 and pattern[j] == text[s + j]:
            j -= 1
        if j < 0:
            return True
        else:
            bc_shift = j - bad_char.get(text[s + j], -1)
            gs_shift = good_suffix[j + 1]
            s += max(1, max(bc_shift, gs_shift))
    return False

# --- Test data ---
text = ("This is a sample text with some vulnerable words like brother and sister " * 5000)
pattern = "brother"

def format_time(elapsed):
    minutes = int(elapsed // 60)
    seconds = int(elapsed % 60)
    milliseconds = (elapsed - int(elapsed)) * 1000
    return f"{minutes}m {seconds}s {milliseconds:.3f}ms"

# Measure Bad Character heuristic
start_bc = time.time()
found_bc = boyer_moore_bad_character(text, pattern)
end_bc = time.time()
elapsed_bc = end_bc - start_bc

# Measure Good Suffix heuristic
start_gs = time.time()
found_gs = boyer_moore_good_suffix(text, pattern)
end_gs = time.time()
elapsed_gs = end_gs - start_gs

# Measure Combined heuristic
start_comb = time.time()
found_comb = boyer_moore_combined(text, pattern)
end_comb = time.time()
elapsed_comb = end_comb - start_comb

# Print results
print("Boyer-Moore Bad Character Heuristic:")
print(f"Pattern: \"{pattern}\"")
print(f"Found: {found_bc}")
print(f"Elapsed time: {format_time(elapsed_bc)}")

print("\nBoyer-Moore Good Suffix Heuristic:")
print(f"Pattern: \"{pattern}\"")
print(f"Found: {found_gs}")
print(f"Elapsed time: {format_time(elapsed_gs)}")

print("\nBoyer-Moore Combined Heuristics:")
print(f"Pattern: \"{pattern}\"")
print(f"Found: {found_comb}")
print(f"Elapsed time: {format_time(elapsed_comb)}")


Boyer-Moore Bad Character Heuristic:
Pattern: "brother"
Found: True
Elapsed time: 0m 0s 0.095ms

Boyer-Moore Good Suffix Heuristic:
Pattern: "brother"
Found: True
Elapsed time: 0m 0s 0.083ms

Boyer-Moore Combined Heuristics:
Pattern: "brother"
Found: True
Elapsed time: 0m 0s 0.058ms
